In [1]:
import os
import time
import uuid
import requests
import base64
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime

# 환경 변수 로드
# load_dotenv()
CLOVA_OCR_SECRET = "R0t3RkxNVmJud2ZmSWZIY3BUR0Z4VlVqQUV5b2dQV00="
OCR_API_URL = "https://3ja254nf6l.apigw.ntruss.com/custom/v1/38065/f6e2a7f6d39340c1a967762f8265e55ed0cf9e441f30ee185ba6a26df73d34db/general"

def call_clova_ocr(image_path):
    try:
        # 이미지 파일 존재 확인
        if not os.path.exists(image_path):
            print(f"Error: 이미지 파일을 찾을 수 없습니다: {image_path}")
            return None

        # 이미지 파일 읽기
        with open(image_path, 'rb') as f:
            image_data = base64.b64encode(f.read()).decode('utf-8')
        
        # OCR API 요청 헤더 설정
        headers = {
            'X-OCR-SECRET': CLOVA_OCR_SECRET,
            'Content-Type': 'application/json'
        }
        
        # CLOVA_OCR_SECRET 확인
        if not CLOVA_OCR_SECRET:
            print("Error: CLOVA_OCR_SECRET이 설정되지 않았습니다.")
            return None
        
        # OCR API 요청 데이터 구성
        request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'prescription',
                    'data': image_data
                }
            ],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000))
        }
        
        # OCR API 호출
        response = requests.post(OCR_API_URL, headers=headers, json=request_json)
        
        # 응답 상태 코드 확인
        if response.status_code != 200:
            print(f"Error: API 호출 실패 (status code: {response.status_code})")
            print("응답:", response.text)
            return None
            
        return response.json()
        
    except Exception as e:
        print(f"Error: OCR API 호출 중 오류 발생: {str(e)}")
        return None

def extract_table_from_ocr(ocr_result):
    table_data = []
    
    for image in ocr_result.get("images", []):
        for table in image.get("tables", []):
            row_dict = {}
            
            for cell in table.get("cells", []):
                cell_text_lines = cell.get("cellTextLines", [])
                if not cell_text_lines:
                    continue
                    
                cell_line = cell_text_lines[0]
                cell_words = cell_line.get("cellWords", [])
                if not cell_words:
                    continue
                    
                cell_text = " ".join([word.get("inferText", "").strip() for word in cell_words])
                vertices = cell_line.get("boundingPoly", {}).get("vertices", [])
                if not vertices:
                    continue
                    
                min_x = min(v.get("x", 0) for v in vertices)
                min_y = min(v.get("y", 0) for v in vertices)
                
                matched_row = None
                for row_y in row_dict.keys():
                    if abs(row_y - min_y) < 20:
                        matched_row = row_y
                        break
                        
                if matched_row is None:
                    matched_row = min_y
                    row_dict[matched_row] = []
                    
                row_dict[matched_row].append((min_x, cell_text))
                
            sorted_rows = []
            for row_y in sorted(row_dict.keys()):
                sorted_row = sorted(row_dict[row_y], key=lambda x: x[0])
                sorted_rows.append([text for _, text in sorted_row])
                
            table_data.extend(sorted_rows)
            
    return pd.DataFrame(table_data)

# 테스트 실행
image_path = r'C:\Users\pc\Desktop\project\icare\BackEndiCare\registerPrescription\testdata\IMG_8355.jpg'
ocr_result = call_clova_ocr(image_path)
table_df = extract_table_from_ocr(ocr_result)

# 결과 출력
print("=== 전체 OCR 결과 ===\n")
print(table_df)

# 필요한 정보 추출
result = {
    "약국명": "",
    "약국주소": "",
    "조제일자": "",
    "총수납금액": "",
    "약품목록": []
}

for _, row in table_df.iterrows():
    row_values = [str(val).strip() for val in row.values]
    row_str = ' '.join(row_values).lower()
    
    if '상호' in row_str:
        result['약국명'] = row.iloc[1] if len(row) > 1 else ""
    
    if '사업장소재지' in row_str:
        result['약국주소'] = row.iloc[1] if len(row) > 1 else ""
    
    if '조제일자' in row_str or '발행일' in row_str:
        result['조제일자'] = row.iloc[1] if len(row) > 1 else ""
    
    if '합계' in row_str:
        amount = row.iloc[1] if len(row) > 1 else "0"
        result['총수납금액'] = ''.join(filter(str.isdigit, str(amount)))
    
    if '약품명' in row_str or '약제명' in row_str:
        medicine = {
            "약품명": row.iloc[1] if len(row) > 1 else "",
            "투약량": row.iloc[2] if len(row) > 2 else "1",
            "투약횟수": row.iloc[3] if len(row) > 3 else "3",
            "투약일수": row.iloc[4] if len(row) > 4 else "3",
        }
        medicine["총투약수"] = int(medicine["투약횟수"]) * int(medicine["투약일수"])
        result['약품목록'].append(medicine)

print("\n=== 추출된 정보 ===\n")
import json
print(json.dumps(result, ensure_ascii=False, indent=2))

=== 전체 OCR 결과 ===

Empty DataFrame
Columns: []
Index: []

=== 추출된 정보 ===

{
  "약국명": "",
  "약국주소": "",
  "조제일자": "",
  "총수납금액": "",
  "약품목록": []
}


In [1]:
import os
import time
import uuid
import requests
import base64
import json
import pandas as pd
from datetime import datetime

# Clova OCR API 설정
CLOVA_OCR_SECRET = "R0t3RkxNVmJud2ZmSWZIY3BUR0Z4VlVqQUV5b2dQV20="
OCR_API_URL = "https://3ja254nf6l.apigw.ntruss.com/custom/v2/38065/f6e2a7f6d39340c1a967762f8265e55ed0cf9e441f30ee185ba6a26df73d34db/general"

def call_clova_ocr(image_path):
    try:
        # 이미지 파일 읽기
        with open(image_path, 'rb') as f:
            image_data = base64.b64encode(f.read()).decode('utf-8')
        
        # OCR API 요청
        timestamp = int(time.time() * 1000)
        request_json = {
            "version": "V2",
            "requestId": str(uuid.uuid4()),
            "timestamp": timestamp,
            
            "lang": "ko",
            "images": [
                {
                    "format": "jpg",
                    "name": "prescription",
                    "data": image_data
                }
            ],
            "enableTableDetection": True
        }
        
        headers = {
            "X-OCR-SECRET": CLOVA_OCR_SECRET,
            "Content-Type": "application/json"
        }
        
        response = requests.post(OCR_API_URL, headers=headers, json=request_json)
        print(f"API 응답 상태 코드: {response.status_code}")
        
        if response.status_code != 200:
            print(f"API 호출 실패: {response.text}")
            return None
            
        return response.json()
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

def extract_table_from_ocr(ocr_result):
    table_data = []
    
    for image in ocr_result.get("images", []):
        for table in image.get("tables", []):
            row_dict = {}
            
            for cell in table.get("cells", []):
                cell_text_lines = cell.get("cellTextLines", [])
                if not cell_text_lines:
                    continue
                
                cell_line = cell_text_lines[0]
                cell_words = cell_line.get("cellWords", [])
                if not cell_words:
                    continue
                
                cell_text = " ".join([word.get("inferText", "").strip() for word in cell_words])
                vertices = cell_line.get("boundingPoly", {}).get("vertices", [])
                if not vertices:
                    continue
                
                min_x = min(v.get("x", 0) for v in vertices)
                min_y = min(v.get("y", 0) for v in vertices)
                
                matched_row = None
                for row_y in row_dict.keys():
                    if abs(row_y - min_y) < 20:
                        matched_row = row_y
                        break
                
                if matched_row is None:
                    matched_row = min_y
                    row_dict[matched_row] = []
                
                row_dict[matched_row].append((min_x, cell_text))
            
            sorted_rows = []
            for row_y in sorted(row_dict.keys()):
                sorted_row = sorted(row_dict[row_y], key=lambda x: x[0])
                sorted_rows.append([text for _, text in sorted_row])
            
            table_data.extend(sorted_rows)
    
    return pd.DataFrame(table_data)

# 테스트 실행
image_path = 'C:/Users/pc/Desktop/project/icare/BackEndiCare/registerPrescription/testdata/IMG_8355.jpg'
print("\n=== OCR 테스트 시작 ===\n")

# OCR API 호출
ocr_result = call_clova_ocr(image_path)
if not ocr_result:
    print("OCR 처리 실패")
    exit()

print("\n=== OCR 원본 응답 ===")
print(json.dumps(ocr_result, ensure_ascii=False, indent=2))

# 테이블 데이터 추출
print("\n=== 테이블 데이터 추출 ===")
table_df = extract_table_from_ocr(ocr_result)
print(table_df)

# 필요한 정보 추출
result = {
    "약국명": "",
    "약국주소": "",
    "조제일자": "",
    "총수납금액": "",
    "약품목록": []
}

# 데이터 처리
for _, row in table_df.iterrows():
    row_values = [str(val).strip() for val in row.values]
    row_str = ' '.join(row_values).lower()
    
    if '상호' in row_str:
        result['약국명'] = row.iloc[1] if len(row) > 1 else ""
    
    if '사업장소재지' in row_str:
        result['약국주소'] = row.iloc[1] if len(row) > 1 else ""
    
    if '조제일자' in row_str or '발행일' in row_str:
        result['조제일자'] = row.iloc[1] if len(row) > 1 else ""
    
    if '합계' in row_str:
        amount = row.iloc[1] if len(row) > 1 else "0"
        result['총수납금액'] = ''.join(filter(str.isdigit, str(amount)))
    
    if '약품명' in row_str or '약제명' in row_str:
        medicine = {
            "약품명": row.iloc[1] if len(row) > 1 else "",
            "투약량": row.iloc[2] if len(row) > 2 else "1",
            "투약횟수": row.iloc[3] if len(row) > 3 else "3",
            "투약일수": row.iloc[4] if len(row) > 4 else "3",
            "총투약수": 0  # 나중에 계산
        }
        try:
            medicine["총투약수"] = int(medicine["투약횟수"]) * int(medicine["투약일수"])
        except:
            medicine["총투약수"] = 9  # 기본값
        result['약품목록'].append(medicine)

print("\n=== 최종 추출 결과 ===")
print(json.dumps(result, ensure_ascii=False, indent=2))


=== OCR 테스트 시작 ===

API 응답 상태 코드: 404
API 호출 실패: {"error":{"errorCode":"300","message":"Not Found Exception","details":"URL not found."}}
OCR 처리 실패

=== OCR 원본 응답 ===
null

=== 테이블 데이터 추출 ===


AttributeError: 'NoneType' object has no attribute 'get'

: 

In [3]:
import os
import json
import requests
import uuid
import time

# 네이버 CLOVA OCR API 정보
API_URL = "https://3ja254nf6l.apigw.ntruss.com/custom/v1/38065/f6e2a7f6d39340c1a967762f8265e55ed0cf9e441f30ee185ba6a26df73d34db/general"
SECRET_KEY = "V01SaWJBVFdjcnlER3NSTmV4dlBFUHZDeFRpWmF4bGk="

def ocr_process(image_path):
    """OCR 기능: 업로드한 이미지 처리"""
    try:
        if os.path.exists(image_path):
            # OCR 요청 데이터 구성
            request_json = {
                "version": "V2",
                "requestId": str(uuid.uuid4()),
                "timestamp": int(round(time.time() * 1000)),
                "lang": "ko",
                "images": [{"format": "jpg", "name": "ocr_image"}]
            }

            # 멀티파트 폼 데이터 구성: 'message' 필드에 JSON, 'file' 필드에 이미지 파일
            files = {
                'message': (None, json.dumps(request_json), 'application/json'),
                'file': (os.path.basename(image_path), open(image_path, "rb"), "image/jpeg")
            }

            headers = {
                "X-OCR-SECRET": SECRET_KEY  # Content-Type은 제거!
            }

            # 네이버 CLOVA OCR API 호출
            response = requests.post(API_URL, headers=headers, files=files)

            if response.status_code == 200:
                ocr_data = response.json()
                extracted_text = [
                    field["inferText"] for field in ocr_data["images"][0]["fields"] if "inferText" in field
                ]

                return {"status": "success", "text": extracted_text}
            else:
                return {"status": "error", "message": "OCR 요청 실패"}
        else:
            return {"status": "error", "message": "파일이 없습니다."}

    except Exception as e:
        return {"status": "error", "message": str(e)}

# 테스트 함수
def test_ocr_process():
    image_path = r"C:\Users\pc\Desktop\project\icare\BackEndiCare\registerPrescription\testdata\IMG_8355.jpg"  # 테스트할 이미지 파일 경로
    result = ocr_process(image_path)
    print(result)

# 테스트 실행
test_ocr_process()

{'status': 'success', 'text': ['약제비', '계산서', '.', '영수증', '(별지제', '11호', '서식)', '영수증번호', '투약일수', '3', '환자성명', '조유림님(980522-2)', '조제일자', '2025-01-02', '☑야간', '□공휴', '약제비총액', '(1+2+3)', '21,370', '원', '본인부담금', '(1)', '6,400', '원', '보험자부담금', '(2)', '14,970', '원', '비급여(전액본인)', '(3)', '0원', '카', '드', '6,400 원', '총수납금액', '현금영수증', '0원', '(1+3)', '현', '금', '0원', '합', '계', '6,400 원', '현금영수증', '신분확인번호', '(', ')', '현금승인번호', '사업자등록번호', '880-13-02118', '서울시', '도봉구', '마들로13길61', '사업장소재지', '씨드큐브 창동 4층', '상', '호', '창동메디컬약국', '성', '명', '김영훈', '발행일', '2025-01-02', '이', '계산서', '/', '영수증은', '소득세법상', '의료비', '또는', '조세특례제한법에', '의한', '현금영수증(현금영수증', '승인번호가', '기재된', '경우)', '공제신청에', '사용할', '수', '있습니다.', '다만,', '지출증빙용으로', '발급된', '현금영', '수중(지출증빙)은', '공제신청에', '사용할', '수', '없습니다.', '·', '이', '계산서', '영수증에', '대한', '세부내역을', '요구할', '수', '있습니다.', '전액본인부담금이란', '국민건강보험법', '시행규칙', '별표', '5의', '규정에', '의한', '요양급여비용의', '본인잔액', '부담항목', '비용을', '말합니다.', '약품명', '투약량', '횟수', '일수', '총투', '펜잘8시간이알서방', '1', '3', '3', '9', '맥스펜정', '1', '

In [7]:
import os
import json
import requests
import uuid
import time
import openai  # GPT API 호출용
from openai import ChatCompletion  # 새 인터페이스 사용

# 네이버 CLOVA OCR API 정보
API_URL = "https://3ja254nf6l.apigw.ntruss.com/custom/v1/38065/f6e2a7f6d39340c1a967762f8265e55ed0cf9e441f30ee185ba6a26df73d34db/general"
SECRET_KEY = "V01SaWJBVFdjcnlER3NSTmV4dlBFUHZDeFRpWmF4bGk="

# OpenAI API 키 (환경 변수 또는 직접 입력)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY


def ocr_process(image_path):
    """OCR 기능: 업로드한 이미지 처리"""
    try:
        if os.path.exists(image_path):
            # OCR 요청 데이터 구성
            request_json = {
                "version": "V2",
                "requestId": str(uuid.uuid4()),
                "timestamp": int(round(time.time() * 1000)),
                "lang": "ko",
                "images": [{"format": "jpg", "name": "ocr_image"}]
            }

            # 멀티파트 폼 데이터 구성: 'message' 필드에 JSON, 'file' 필드에 이미지 파일
            files = {
                'message': (None, json.dumps(request_json), 'application/json'),
                'file': (os.path.basename(image_path), open(image_path, "rb"), "image/jpeg")
            }

            headers = {
                "X-OCR-SECRET": SECRET_KEY  # Content-Type은 제거!
            }

            # 네이버 CLOVA OCR API 호출
            response = requests.post(API_URL, headers=headers, files=files)

            if response.status_code == 200:
                ocr_data = response.json()
                # OCR API의 응답 구조에 따라 필드 추출 (예제에서는 "fields" 리스트 내 "inferText" 값들을 사용)
                extracted_text = [
                    field["inferText"] for field in ocr_data["images"][0]["fields"] if "inferText" in field
                ]
                return {"status": "success", "text": extracted_text}
            else:
                return {"status": "error", "message": "OCR 요청 실패"}
        else:
            return {"status": "error", "message": "파일이 없습니다."}

    except Exception as e:
        return {"status": "error", "message": str(e)}


def extract_info_with_gpt(ocr_text_list, prescription_number):
    """
    OCR로 추출한 텍스트 리스트와 처방전번호를 받아서 GPT에 전달하여
    약국명, 조제일자, 약국주소, 총수납금액, 약품목록 등의 정보를 JSON으로 반환받는다.
    """
    # OCR 결과 리스트를 하나의 문자열로 결합
    table_text = " ".join(ocr_text_list)
    
    prompt = f"""
다음은 처방전에서 OCR로 추출한 텍스트입니다:
{table_text}

위 텍스트에서 다음 정보를 추출하여 JSON 형식으로 반환해주세요:
1. 약국명 (상호 다음에 나오는 이름)
2. 처방전번호: "{prescription_number}"
3. 조제일자: 조제일자 또는 발행일 다음에 나오는 날짜
4. 약국주소: 사업장소재지 다음에 나오는 주소
5. 총수납금액: 총수납금액 또는 합계 다음에 나오는 금액 (숫자만)
6. 약품목록: 약품명, 투약량, 투약횟수, 투약일수가 포함된 행들

출력은 반드시 아래 JSON 형식으로 해주세요:
{{
    "약국명": "찾은 약국명",
    "처방전번호": "{prescription_number}",
    "조제일자": "찾은 날짜",
    "약국주소": "찾은 주소",
    "총수납금액": "찾은 금액",
    "약품목록": [
        {{
            "약품명": "약품명",
            "투약량": "투약량",
            "투약횟수": "투약횟수",
            "투약일수": "투약일수"
        }}
    ]
}}
    """

    # GPT API 호출 (GPT-4 사용)
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )

    gpt_response = response["choices"][0]["message"]["content"].strip()
    try:
        result = json.loads(gpt_response)
    except Exception as e:
        raise ValueError(f"GPT 응답 JSON 파싱 오류: {e}\n응답 내용: {gpt_response}")
    return result


# 테스트 함수
def test_ocr_and_gpt():
    image_path = r"C:\Users\pc\Desktop\project\icare\BackEndiCare\registerPrescription\testdata\IMG_8355.jpg"  # 테스트 이미지 파일 경로
    ocr_result = ocr_process(image_path)
    
    if ocr_result["status"] == "success":
        ocr_text_list = ocr_result["text"]
        # 처방전번호 생성 (예시로 UUID 앞 8자리 사용)
        prescription_number = f"RX-{uuid.uuid4().hex[:8]}"
        
        try:
            gpt_result = extract_info_with_gpt(ocr_text_list, prescription_number)
            print("GPT 결과:")
            print(json.dumps(gpt_result, ensure_ascii=False, indent=4))
        except Exception as e:
            print("GPT 호출 중 오류 발생:", e)
    else:
        print("OCR 오류:", ocr_result["message"])


# 테스트 실행
if __name__ == "__main__":
    test_ocr_and_gpt()


GPT 호출 중 오류 발생: 'ChatCompletion' object is not subscriptable


In [9]:
import os
import json
import requests
import uuid
import time
import openai  # GPT API 호출용
from openai import ChatCompletion  # 새 인터페이스 사용

# 네이버 CLOVA OCR API 정보
API_URL = "https://3ja254nf6l.apigw.ntruss.com/custom/v1/38065/f6e2a7f6d39340c1a967762f8265e55ed0cf9e441f30ee185ba6a26df73d34db/general"
SECRET_KEY = "V01SaWJBVFdjcnlER3NSTmV4dlBFUHZDeFRpWmF4bGk="

# OpenAI API 키 (환경 변수 또는 직접 입력)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY


def ocr_process(image_path):
    """OCR 기능: 업로드한 이미지 처리"""
    try:
        if os.path.exists(image_path):
            # OCR 요청 데이터 구성
            request_json = {
                "version": "V2",
                "requestId": str(uuid.uuid4()),
                "timestamp": int(round(time.time() * 1000)),
                "lang": "ko",
                "images": [{"format": "jpg", "name": "ocr_image"}]
            }

            # 멀티파트 폼 데이터 구성: 'message' 필드에 JSON, 'file' 필드에 이미지 파일
            files = {
                'message': (None, json.dumps(request_json), 'application/json'),
                'file': (os.path.basename(image_path), open(image_path, "rb"), "image/jpeg")
            }

            headers = {
                "X-OCR-SECRET": SECRET_KEY  # Content-Type은 제거!
            }

            # 네이버 CLOVA OCR API 호출
            response = requests.post(API_URL, headers=headers, files=files)

            if response.status_code == 200:
                ocr_data = response.json()
                # OCR API의 응답 구조에 따라 필드 추출 (예제에서는 "fields" 리스트 내 "inferText" 값들을 사용)
                extracted_text = [
                    field["inferText"] for field in ocr_data["images"][0]["fields"] if "inferText" in field
                ]
                return {"status": "success", "text": extracted_text}
            else:
                return {"status": "error", "message": "OCR 요청 실패"}
        else:
            return {"status": "error", "message": "파일이 없습니다."}

    except Exception as e:
        return {"status": "error", "message": str(e)}


def extract_info_with_gpt(ocr_text_list, prescription_number):
    """
    OCR로 추출한 텍스트 리스트와 처방전번호를 받아서 GPT에 전달하여
    약국명, 조제일자, 약국주소, 총수납금액, 약품목록 등의 정보를 JSON으로 반환받는다.
    """
    # OCR 결과 리스트를 하나의 문자열로 결합
    table_text = " ".join(ocr_text_list)
    
    prompt = f"""
        다음은 처방전에서 OCR로 추출한 텍스트입니다:
        {table_text}

        위 텍스트에서 다음 정보를 추출하여 JSON 형식으로 반환해주세요:
        1. 약국명 (상호 다음에 나오는 이름)
        2. 처방전번호: "{prescription_number}"
        3. 조제일자: 조제일자 또는 발행일 다음에 나오는 날짜
        4. 약국주소: 사업장소재지 다음에 나오는 주소
        5. 총수납금액: 합계 다음에 나오는 금액 (숫자만)
        6. 약품목록: 약품명, 투약량, 투약횟수, 투약일수가 포함된 행들

        출력은 반드시 아래 JSON 형식으로 해주세요
        JSON 형식을 제외한 어느 말도 하지 마세요.
        
        :
        {{
            "약국명": "찾은 약국명",
            "처방전번호": "{prescription_number}",
            "조제일자": "찾은 날짜",
            "약국주소": "찾은 주소",
            "총수납금액": "찾은 금액",
            "약품목록": [
                {{
                    "약품명": "약품명",
                    "투약량": "투약량",
                    "투약횟수": "투약횟수",
                    "투약일수": "투약일수"
                }}
            ]
        }}
    """

    # GPT API 호출 (GPT-4 사용)
    client = openai.OpenAI(api_key=OPENAI_API_KEY)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )

    gpt_response = response.choices[0].message.content.strip()
    try:
        result = json.loads(gpt_response)
    except Exception as e:
        raise ValueError(f"GPT 응답 JSON 파싱 오류: {e}\n응답 내용: {gpt_response}")
    return result


# 테스트 함수
def test_ocr_and_gpt():
    image_path = r"C:\Users\pc\Desktop\project\icare\BackEndiCare\registerPrescription\testdata\IMG_8355.jpg"  # 테스트 이미지 파일 경로
    ocr_result = ocr_process(image_path)
    
    if ocr_result["status"] == "success":
        ocr_text_list = ocr_result["text"]
        # 처방전번호 생성 (예시로 UUID 앞 8자리 사용)
        prescription_number = f"RX-{uuid.uuid4().hex[:8]}"
        
        try:
            gpt_result = extract_info_with_gpt(ocr_text_list, prescription_number)
            print("GPT 결과:")
            print(json.dumps(gpt_result, ensure_ascii=False, indent=4))
        except Exception as e:
            print("GPT 호출 중 오류 발생:", e)
    else:
        print("OCR 오류:", ocr_result["message"])


# 테스트 실행
if __name__ == "__main__":
    test_ocr_and_gpt()


GPT 호출 중 오류 발생: GPT 응답 JSON 파싱 오류: Expecting value: line 1 column 1 (char 0)
응답 내용: ```json
{
    "약국명": "창동메디컬약국",
    "처방전번호": "RX-9d95c27f",
    "조제일자": "2025-01-02",
    "약국주소": "서울시 도봉구 마들로13길61 씨드큐브 창동 4층",
    "총수납금액": "6400",
    "약품목록": [
        {
            "약품명": "펜잘8시간이알서방",
            "투약량": "1",
            "투약횟수": "3",
            "투약일수": "3"
        },
        {
            "약품명": "맥스펜정",
            "투약량": "1",
            "투약횟수": "3",
            "투약일수": "3"
        },
        {
            "약품명": "케이캡정50mg",
            "투약량": "1",
            "투약횟수": "1",
            "투약일수": "3"
        },
        {
            "약품명": "코대원정",
            "투약량": "1",
            "투약횟수": "3",
            "투약일수": "3"
        },
        {
            "약품명": "뮤코스텐캡슐",
            "투약량": "1",
            "투약횟수": "3",
            "투약일수": "3"
        },
        {
            "약품명": "세악틸정",
            "투약량": "1",
            "투약횟수": "2",
            "투약일수": "3"
        },
        {
     